In [69]:
import xgboost as xgb
import tqdm as tq
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
from tqdm import tqdm
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

### Train Data Preprocessing

In [59]:
train = pd.read_csv('train.csv')

In [69]:
train.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [60]:
train.pop('일조(hr)')
train.pop('일사(MJ/m2)')

train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')
train['month'] = train['일시'].dt.month
train['day'] = train['일시'].dt.weekday
train['hour'] = train['일시'].dt.hour
train['holiday'] = train.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
train['sin_time'] = np.sin(2*np.pi*train.hour/24) # cyclical encoding
train['cos_time'] = np.cos(2*np.pi*train.hour/24)
train['DI'] = 1.8*train['기온(C)'] - 0.55*(1-train['습도(%)'])*(1.8*train['기온(C)']-26) + 32 # 불쾌지수

train.pop('num_date_time')
train.pop('일시')
train.pop('hour')
train['power'] = train['전력소비량(kWh)']
train.pop('전력소비량(kWh)')
train['강수량(mm)'] = train['강수량(mm)'].fillna(0)
train['풍속(m/s)'] = train['풍속(m/s)'].fillna(0)
train['습도(%)'] = train['습도(%)'].fillna(0)

In [71]:
train.to_csv('train_preprocessed.csv', index=False)

In [72]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,holiday,sin_time,cos_time,DI,power
0,1,18.6,0.0,0.9,42.0,6,2,0,0.000000,1.000000,234.154,1085.28
1,1,18.0,0.0,1.1,45.0,6,2,0,0.258819,0.965926,219.280,1047.36
2,1,17.7,0.0,1.5,45.0,6,2,0,0.500000,0.866025,205.672,974.88
3,1,16.7,0.0,1.4,48.0,6,2,0,0.707107,0.707107,167.011,953.76
4,1,18.4,0.0,2.8,43.0,6,2,0,0.866025,0.500000,229.592,986.40


### Test Data Preprocessing

In [61]:
test = pd.read_csv('test.csv')

In [62]:
test['일시'] = pd.to_datetime(test['일시'], format='%Y%m%d %H')
test['month'] = test['일시'].dt.month
test['day'] = test['일시'].dt.weekday
test['hour'] = test['일시'].dt.hour
test['holiday'] = test.apply(lambda x: 0 if x['day']<5 else 1, axis=1)
test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)
test['DI'] = 1.8*test['기온(C)'] - 0.55*(1-test['습도(%)'])*(1.8*test['기온(C)']-26) + 32

test.pop('num_date_time')
test.pop('일시')
test.pop('hour')

0         0
1         1
2         2
3         3
4         4
         ..
16795    19
16796    20
16797    21
16798    22
16799    23
Name: hour, Length: 16800, dtype: int64

In [75]:
test.to_csv('test_preprocessed.csv', index=False)

In [14]:
test.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,holiday,sin_time,cos_time,DI
0,1,23.5,0.0,2.2,72,8,3,0,0.000000,1.000000,710.815
1,1,23.0,0.0,0.9,72,8,3,0,0.258819,0.965926,674.770
2,1,22.7,0.0,1.5,75,8,3,0,0.500000,0.866025,677.662
3,1,22.1,0.0,1.3,78,8,3,0,0.707107,0.707107,655.363
4,1,21.8,0.0,1.0,77,8,3,0,0.866025,0.500000,624.672


### Training

In [63]:
import os
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split

In [64]:
def SMAPE(true, pred):
    return np.mean((np.abs(true - pred))/(np.abs(true) + np.abs(pred))) * 100    

In [20]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,week,holiday,sin_time,cos_time,DI,power
0,1,18.6,0.0,0.9,42.0,6,2,22,0,0.000000,1.000000,234.154,1085.28
1,1,18.0,0.0,1.1,45.0,6,2,22,0,0.258819,0.965926,219.280,1047.36
2,1,17.7,0.0,1.5,45.0,6,2,22,0,0.500000,0.866025,205.672,974.88
3,1,16.7,0.0,1.4,48.0,6,2,22,0,0.707107,0.707107,167.011,953.76
4,1,18.4,0.0,2.8,43.0,6,2,22,0,0.866025,0.500000,229.592,986.40


In [65]:
def tr_ts_split(train, test, number, val_hour):
    x = train.loc[train['건물번호']==number].drop(['건물번호','power'], axis=1)
    y = train.loc[train['건물번호']==number, 'power']

    #if number == 54:
    x = pd.concat([x[:601], x[700:]], axis=0).reset_index(drop=True)
    y = pd.concat([y[:601], y[700:]]).reset_index(drop=True)

    if val_hour == 0:
        return x, None, y, None
    else:
        x_train, x_valid, y_train, y_valid = x[:-val_hour], x[-val_hour:], y[:-val_hour], y[-val_hour:]
        return x_train, x_valid.reset_index(drop=True), y_train, y_valid.reset_index(drop=True)

In [66]:
def fit_and_pred_XGB(train, test, number, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, number, 7*24)

    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, seed=seed,
                            gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds=300, verbose=False)
    pred = xgb_reg.predict(x_valid)
    pred = pd.Series(pred)
    
    x_test = test[test['건물번호']==number]
    y_test_pred = xgb_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred

In [74]:
def fit_and_pred_RF(train, test, number, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train.dropna(), test, number, 7*24)
    
    rf_reg = RandomForestRegressor(random_state=seed)
    rf_reg.fit(x_train, y_train)
    pred = rf_reg.predict(x_valid)
    pred = pd.Series(pred)

    x_test = test[test['건물번호']==number]
    y_test_pred = rf_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred                                    

In [75]:
pred_val_ens= []
test_ens = []

summary_list = []
ans_val_list = []
pred_val_list = []
pred_test_list = []

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    summary_dict = {}

    smape_val, r2_val, ans_val, pred_val, pred_test = fit_and_pred_RF(train, test, i+1, seed=0)

    summary_dict['건물번호'] = i+1
    summary_dict['SMAPE'] = smape_val
    summary_dict['R2'] = r2_val
    summary_list.append(summary_dict)
    ans_val_list.append(ans_val)
    pred_val_list.append(pred_val)
    pred_test_list.append(pred_test)

ans_val_whole = np.concatenate(ans_val_list)
pred_val_whole = np.concatenate(pred_val_list)
pred_test_whole = np.concatenate(pred_test_list)
smape_val =SMAPE(ans_val_whole, pred_val_whole)

  0%|          | 0/100 [00:00<?, ?it/s]

===== < BUILDING NO.1 > =====


  1%|          | 1/100 [00:00<00:47,  2.10it/s]

SMAPE: 3.81607975204115
R2: 0.9382542184288511
===== < BUILDING NO.2 > =====


  2%|▏         | 2/100 [00:00<00:46,  2.09it/s]

...

SMAPE: 7.076964088552592
R2: 0.8518380869876305
===== < BUILDING NO.99 > =====


 99%|█████████▉| 99/100 [00:45<00:00,  2.17it/s]

SMAPE: 1.6628715344759977
R2: 0.978905208400493
===== < BUILDING NO.100 > =====


100%|██████████| 100/100 [00:46<00:00,  2.15it/s]

SMAPE: 2.9758569305188023
R2: 0.958168573833171


In [76]:
summary_list

[{'건물번호': 1, 'SMAPE': 3.81607975204115, 'R2': 0.9382542184288511},
 {'건물번호': 2, 'SMAPE': 4.867200648713607, 'R2': 0.7763007096473291},
 {'건물번호': 3, 'SMAPE': 7.7069141261757705, 'R2': 0.7044058382387558},
 {'건물번호': 4, 'SMAPE': 2.6295317324970036, 'R2': 0.9541398097540355},
 {'건물번호': 5, 'SMAPE': 2.6806479501464584, 'R2': 0.9727180063840729},
 {'건물번호': 6, 'SMAPE': 2.16355388770827, 'R2': 0.9704339792850838},
 {'건물번호': 7, 'SMAPE': 4.631251200590728, 'R2': 0.8938503046146825},
 {'건물번호': 8, 'SMAPE': 2.6015812045585722, 'R2': 0.7028133728965408},
 {'건물번호': 9, 'SMAPE': 2.1283778181013053, 'R2': 0.9665456168458111},
 {'건물번호': 10, 'SMAPE': 4.690170995407238, 'R2': 0.06500874864653094},
 {'건물번호': 11, 'SMAPE': 1.9116744901547011, 'R2': 0.8832901180685022},
 {'건물번호': 12, 'SMAPE': 2.4815619328076512, 'R2': 0.8664673302670154},
 {'건물번호': 13, 'SMAPE': 3.2920364666488298, 'R2': 0.28128614925023554},
 {'건물번호': 14, 'SMAPE': 8.470760739573072, 'R2': 0.22610613234099208},
 {'건물번호': 15, 'SMAPE': 1.443975229

In [77]:
smape_val

2.8263124491815237

In [78]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = pred_test_whole
sub.to_csv('rf_sub_23.csv', index=False)

### Improvement

In [67]:
train['적정온도차이'] = train['기온(C)'] - 26

In [68]:
test['적정온도차이'] = test['기온(C)'] - 26

In [34]:
train.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,holiday,sin_time,cos_time,DI,power,적정온도차이
0,1,18.6,0.0,0.9,42.0,6,2,0,0.000000,1.000000,234.154,1085.28,-7.4
1,1,18.0,0.0,1.1,45.0,6,2,0,0.258819,0.965926,219.280,1047.36,-8.0
2,1,17.7,0.0,1.5,45.0,6,2,0,0.500000,0.866025,205.672,974.88,-8.3
3,1,16.7,0.0,1.4,48.0,6,2,0,0.707107,0.707107,167.011,953.76,-9.3
4,1,18.4,0.0,2.8,43.0,6,2,0,0.866025,0.500000,229.592,986.40,-7.6


In [37]:
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [38]:
def fit_and_pred_XGB_with_weight(train, test, number, weight, seed=0):
    x_train, x_valid, y_train, y_valid = tr_ts_split(train, test, number, 7*24)

    xgb_reg = XGBRegressor(n_estimators=10000, eta=0.01, seed=seed,
                            gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
    xgb_reg.set_params(**{'objective':weighted_mse(weight)})
    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)],
                early_stopping_rounds=300, verbose=False)
    pred = xgb_reg.predict(x_valid)
    pred = pd.Series(pred)
    
    x_test = test[test['건물번호']==number]
    y_test_pred = xgb_reg.predict(x_test.drop('건물번호', axis=1))

    smape_val = SMAPE(y_valid, pred)
    r2_val = sklearn.metrics.r2_score(y_valid, pred)
    print(f'SMAPE: {smape_val}')
    print(f'R2: {r2_val}')
    return smape_val, r2_val, y_valid, pred, y_test_pred

In [44]:
pred_val_ens= []
test_ens = []

summary_list = []
ans_val_list = []
pred_val_list = []
pred_test_list = []

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    summary_dict = {}

    smape_val, r2_val, ans_val, pred_val, pred_test = fit_and_pred_XGB_with_weight(train, test, i+1, 100, seed=0)

    summary_dict['건물번호'] = i+1
    summary_dict['SMAPE'] = smape_val
    summary_dict['R2'] = r2_val
    summary_list.append(summary_dict)
    ans_val_list.append(ans_val)
    pred_val_list.append(pred_val)
    pred_test_list.append(pred_test)

ans_val_whole = np.concatenate(ans_val_list)
pred_val_whole = np.concatenate(pred_val_list)
pred_test_whole = np.concatenate(pred_test_list)
smape_val =SMAPE(ans_val_whole, pred_val_whole)

  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


===== < BUILDING NO.1 > =====


  1%|          | 1/100 [00:01<02:57,  1.80s/it]

SMAPE: 3.883634407146668
R2: 0.9428148434678705
===== < BUILDING NO.2 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
  2%|▏         | 2/100 [00:03<02:26,  1.50s/it]

...

SMAPE: 7.038908245544515
R2: 0.8492584312608136
===== < BUILDING NO.99 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
 99%|█████████▉| 99/100 [03:00<00:01,  1.69s/it]

SMAPE: 1.6848495597962838
R2: 0.9786802231329808
===== < BUILDING NO.100 > =====


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
100%|██████████| 100/100 [03:02<00:00,  1.82s/it]

SMAPE: 2.642659282172129
R2: 0.9674120094300811


In [45]:
summary_list

[{'건물번호': 1, 'SMAPE': 3.883634407146668, 'R2': 0.9428148434678705},
 {'건물번호': 2, 'SMAPE': 5.256542311528698, 'R2': 0.8268383595313882},
 {'건물번호': 3, 'SMAPE': 5.545908115619256, 'R2': 0.8225081381620768},
 {'건물번호': 4, 'SMAPE': 2.0216084532929974, 'R2': 0.9738995748382593},
 {'건물번호': 5, 'SMAPE': 2.8028523787496087, 'R2': 0.9733667899879699},
 {'건물번호': 6, 'SMAPE': 1.861585193427125, 'R2': 0.981714605738076},
 {'건물번호': 7, 'SMAPE': 5.0474765252492375, 'R2': 0.8962074296471658},
 {'건물번호': 8, 'SMAPE': 2.5489555005401963, 'R2': 0.7355127209176974},
 {'건물번호': 9, 'SMAPE': 2.0478630548996173, 'R2': 0.975397976943655},
 {'건물번호': 10, 'SMAPE': 3.907172078032968, 'R2': 0.32611646656584903},
 {'건물번호': 11, 'SMAPE': 1.676392266013526, 'R2': 0.9014899611347567},
 {'건물번호': 12, 'SMAPE': 1.8723574629050932, 'R2': 0.9008362680544917},
 {'건물번호': 13, 'SMAPE': 3.0274657714415754, 'R2': 0.3797024654736437},
 {'건물번호': 14, 'SMAPE': 8.411099584140395, 'R2': 0.2536910586619444},
 {'건물번호': 15, 'SMAPE': 1.254851791649

In [46]:
smape_val

2.638455250836857

In [47]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = pred_test_whole
sub.to_csv('xgb_sub_21.csv', index=False)

### Parameter Tuning - Grid Search CV

In [48]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [ ]:
df = pd.DataFrame(columns = ['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split'])
preds = np.array([])
grid = {
    'n_estimators': [30, 50, 70, 100, 150],
    'max_depth': [6, 8, 10, 12],
    'min_samples_leaf': [8, 12, 18],
    'min_samples_split': [8, 16, 20]
}

for i in tqdm(range(100)):
    print(f"===== < BUILDING NO.{i+1} > =====")
    x_train, x_valid, y_train, y_valid = tr_ts_split(train.dropna(), test, i+1, 7*24)
    gcv = GridSearchCV(estimator=RandomForestRegressor(random_state=0),
                       param_grid=grid,
                       scoring=make_scorer(SMAPE, greater_is_better=False),
                       cv=5,
                       refit=True,
                       verbose=True)
    gcv.fit(x_train, y_train)
    best = gcv.best_estimator_
    params = gcv.best_params_
    print(params)
    pred = best.predict(x_valid)
    print(f'SMAPE: {SMAPE(y_valid, pred)}')
    preds = np.append(preds, pred)
    df = pd.concat([df, pd.DataFrame(params, index=[0])], axis=0)

  0%|          | 0/100 [00:00<?, ?it/s]

===== < BUILDING NO.1 > =====
Fitting 5 folds for each of 180 candidates, totalling 900 fits


  1%|          | 1/100 [02:30<4:08:37, 150.69s/it]

{'max_depth': 8, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 150}
SMAPE: 4.2450750272584745
===== < BUILDING NO.2 > =====
Fitting 5 folds for each of 180 candidates, totalling 900 fits


  2%|▏         | 2/100 [05:03<4:08:03, 151.87s/it]

...

{'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 70}
SMAPE: 3.1944882260505283
===== < BUILDING NO.93 > =====
Fitting 5 folds for each of 180 candidates, totalling 900 fits


 93%|█████████▎| 93/100 [3:50:48<17:37, 151.02s/it]

{'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 50}
SMAPE: 3.3251798994406743
===== < BUILDING NO.94 > =====
Fitting 5 folds for each of 180 candidates, totalling 900 fits


 94%|█████████▍| 94/100 [3:53:17<15:01, 150.33s/it]

{'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 150}
SMAPE: 3.0058307993349924
===== < BUILDING NO.95 > =====
Fitting 5 folds for each of 180 candidates, totalling 900 fits


In [133]:
df.to_csv('hyperparameter_xgb.csv', index=False)

In [84]:
rf_params = df

In [85]:
rf_params.head()

,n_estimators,max_depth,min_samples_leaf,min_samples_split
0,150,8,8,8
0,100,8,8,8
0,30,6,8,8
0,100,8,8,8
0,50,10,8,8


In [90]:
preds = np.array([])
for i in tqdm(range(100)):
    pred_df = pd.DataFrame()
    for seed in range(0, 6):
        x_train = train.dropna().loc[train['건물번호']==i+1,].drop(['건물번호', 'power'], axis=1)
        y_train = train.dropna().loc[train['건물번호']==i+1, 'power']
        x_test = test.loc[test['건물번호']==i+1,].drop('건물번호', axis=1)

        rf = RandomForestRegressor(random_state=seed, n_estimators=rf_params.iloc[i, 0],
                                   max_depth=rf_params.iloc[i, 1],
                                   min_samples_leaf=rf_params.iloc[i, 2],
                                   min_samples_split=rf_params.iloc[i, 3])

        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        pred_df.loc[:, seed] = y_pred

    pred = pred_df.mean(axis=1)
    preds = np.append(preds, pred)

  8%|▊         | 8/100 [00:15<03:02,  1.98s/it]


IndexError: index 8 is out of bounds for axis 0 with size 8

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['answer'] = preds
sub.to_csv('rf_sub_24.csv', index=False)